In [100]:
from nltk.corpus import stopwords
import nltk
stopword=nltk.corpus.stopwords.words('english')
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [117]:
r=open("sadf.txt").read() # a file that contains random text messages
import pandas as pd
d=pd.read_csv('sadf.txt', sep= '\n',names=['body_text'],header=None)
d.head()

,body_text
0,Importing the libraries that we are going to ...
1,We will save our trained model to the disk usi...
2,"In our case,"
3,"Here, we have bounded /api with the method pre..."
4,We can again load the model by the following m...


In [124]:
p={}
type(p)

dict

In [136]:
def analyser(r,d,p):
   from nltk import tokenize
   s=tokenize.sent_tokenize(r)
   p['Sentences']=s  

   import summa
   from summa import summarizer
   summary=(summarizer.summarize(r))
   p['Summary']=summary #1
    
   noun=[]
   import nltk
   for sentence in s:
    for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             noun.append(word) 
   p['Nouns']=noun[0:10] #2
   verb=[]
   for sentence in s:
       for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
           if (pos == 'VB' or pos == 'VDB' or pos == 'VBG' or pos == 'VBN' or pos =='VBP' or pos =='VBZ'):
               verb.append(word)
   p['Verbs']=verb[0:10]  #3

   def rem_p(text):
       text_nop="".join([char for char in text if char not in string.punctuation])
       return text_nop

   d['body_text_clean']=d['body_text'].apply(lambda x: rem_p(x))

   import re
   def tokenize(text):
       tokens=re.split('\W+',text)
       return tokens
   d['body_text_tokenized']=d['body_text_clean'].apply(lambda x:tokenize(x.lower()))
    
   def remove_stopwords(tokenized_list):
       text=[word for word in tokenized_list if word not in stopword]
       return text
   d['body_text_nostop']=d['body_text_tokenized'].apply(lambda x:remove_stopwords(x))    

   ps=nltk.PorterStemmer()

   def stemming(tokenized_text):
       text=[ps.stem(word)for word in tokenized_text]
       return text
   d['body_text_stemmed']=d['body_text_nostop'].apply(lambda x:stemming(x))
   st=d['body_text_stemmed'].tolist()
   p['Stems']=st[0:1] 

   wn=nltk.WordNetLemmatizer()
   def lemmatizing(tokenized_text):
       text=[wn.lemmatize(word) for word in tokenized_text]
       return text
   d['body_text_lemmatized']=d['body_text_nostop'].apply(lambda x:lemmatizing(x))
   l=d['body_text_lemmatized'].tolist()
   p['Lemma']=l[0:1]





   return p

In [137]:
analyser(r,d,p)

{'Sentences': ['\x7fImporting the libraries that we are going to use to develop our model.',
  'numpy and pandas to manipulate the matrices and data respectively, sklearn.model_selection for splitting data into train and test set and sklearn.linear_model to train our model using LinearRegression.',
  'pickle to save our trained model to the disk, requests to send requests to the server and json to print the result in our terminal.',
  'We will save our trained model to the disk using the pickle library.',
  'Pickle is used to serializing and de-serializing a Python object structure.',
  'In which python object is converted into the byte stream.',
  'dump() method dumps the object into the file specified in the arguments.',
  'In our case, \nHere, we have bounded /api with the method predict().',
  'In which predict method gets the data from the json passed by the requestor.',
  'model.predict() method takes input from the json and converts it into 2D numpy array the results are stored 

In [135]:
import json
with open('result.json', 'w') as fp:
    json.dump(p, fp)

In [66]:
from sklearn.externals import joblib
joblib.dump(p,'NB_spam_model.pkl')

['NB_spam_model.pkl']